In [25]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
import pandas as pd
ticker = 'TCS'
stock_data = yf.download(ticker, start='2023-01-01', end='2023-10-01')
fig = go.Figure(data=[go.Candlestick(x=stock_data.index,open=stock_data['Open'],high=stock_data['High'],low=stock_data['Low'],close=stock_data['Close'])])
# customizing chart
fig.update_layout(
    title=f'{ticker} Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',
    xaxis=dict(rangeslider=dict(visible=True)))# Dark theme
# display
fig.show()


[*********************100%%**********************]  1 of 1 completed


In [26]:
# simple moving average
stock_data['SMA20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['SMA50'] = stock_data['Close'].rolling(window=50).mean()

# adding sma to candlestick
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['SMA20'], mode='lines', name='20-Day SMA'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['SMA50'], mode='lines', name='50-Day SMA'))
fig.update_layout(
    title=f'{ticker} Candlestick Chart with Moving Averages',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',  # Optional: Dark theme for the chart
    xaxis=dict(rangeslider=dict(visible=True))  # Optional: Hide the range slider
)

fig.show()


In [27]:
#analysing specific region
h_start = '2023-05-01'
h_end = '2023-05-05'
fig.add_vrect(x0=h_start, x1=h_end, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)
fig.update_layout(title=f'{ticker} Candlestick Chart with Highlighted Range', xaxis_title='Date', yaxis_title='Price',template='plotly_dark',  # Optional: Dark theme for the chart
    xaxis=dict(rangeslider=dict(visible=True)))
fig.show()


In [28]:
# Add annotation for a specific date
fig.add_annotation(x='2023-07-28', y=stock_data['Close']['2023-07-28'],
                   text="Earnings Release", showarrow=True, arrowhead=1)
fig.update_layout(title=f'{ticker} Candlestick Chart with Annotations', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [29]:
# boilenger band
stock_data['20 Day MA'] = stock_data['Close'].rolling(window=20).mean()
stock_data['20 Day STD'] = stock_data['Close'].rolling(window=20).std()
stock_data['Upper Band'] = stock_data['20 Day MA'] + (stock_data['20 Day STD'] * 2)
stock_data['Lower Band'] = stock_data['20 Day MA'] - (stock_data['20 Day STD'] * 2)
# bbin candlestick
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Upper Band'], mode='lines', name='Upper Bollinger Band'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Lower Band'], mode='lines', name='Lower Bollinger Band'))
#plot
fig.update_layout(title=f'{ticker} Candlestick Chart with Bollinger Bands', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [30]:
# VWAP
stock_data['VWAP'] = (stock_data['Close'] * stock_data['Volume']).cumsum() / stock_data['Volume'].cumsum()
#VWAP in candlestick
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['VWAP'], mode='lines', name='VWAP', line=dict(color='orange')))
fig.update_layout(title=f'{ticker} Candlestick Chart with VWAP', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [31]:
# Calculate MACD and Signal Line
stock_data['EMA_12'] = stock_data['Close'].ewm(span=12, adjust=False).mean()
stock_data['EMA_26'] = stock_data['Close'].ewm(span=26, adjust=False).mean()
stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']
stock_data['Signal_Line'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='red')))

# Customize the layout
fig.update_layout(
    title=f'{ticker} Candlestick Chart with MACD and Signal Line',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',  # Dark theme
    xaxis=dict(rangeslider=dict(visible=True)),
)

# Show the chart
fig.show()

In [33]:
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avggain = pd.Series(gain).rolling(window=window).mean()
    avgloss = pd.Series(loss).rolling(window=window).mean()

    rs = avggain / avgloss
    rsi = 100 - (100 / (1 + rs))

    return rsi

stock_data['RSI_14'] = calculate_rsi(stock_data)
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['RSI_14'],
                         mode='lines', name='RSI 14', yaxis='y2', line=dict(color='purple')))

# Add horizontal lines for RSI thresholds
fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought", yref='y2')
fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold", yref='y2')

# Update layout to include a secondary y-axis for RSI
fig.update_layout(
    title=f'{ticker} Candlestick Chart with RSI',
    xaxis_title='Date',
    yaxis_title='Price',
    yaxis2=dict(title='RSI', overlaying='y', side='right', range=[0, 100]),
    template='plotly_dark',
    xaxis=dict(rangeslider=dict(visible=False))
)

# Show the chart
fig.show()

In [35]:
fig.update_layout(
    title=f'{ticker} Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',  # Dark theme
    xaxis=dict(rangeslider=dict(visible=True)),
)

fig.show()
